In [ ]:
!pip install openai pandas openpyxl


In [ ]:
# ====== Cell 2: Azure GPT-4o API Setup ======
import json
from openai import AzureOpenAI

# ---------- CONFIG ----------
GPT4O_DEPLOYMENT_NAME = "gpt-4o"
GPT4O_AZURE_ENDPOINT = "https://tcoeaiteamgpt4o.openai.azure.com/"
GPT4O_OPENAI_API_KEY = "6ffhCm6wvgxD2LRD7wNZI5sHgqHn4lqYOY7xn8Ycdg7vHvZ8qyujJQQJ99BCACYeBjFXJ3w3AAABACOGwuFA"
GPT4O_OPENAI_API_VERSION = "2024-12-01-preview"

# ---------- CLIENT ----------
client = AzureOpenAI(
    azure_endpoint=GPT4O_AZURE_ENDPOINT,
    api_key=GPT4O_OPENAI_API_KEY,
    api_version=GPT4O_OPENAI_API_VERSION
)

print("✅ Azure GPT-4o client initialized successfully!")


✅ Azure GPT-4o client initialized successfully!


In [ ]:
# ====== Cell 3: Load JSON Dataset ======
import json
import pandas as pd

# Path to your dataset file
file_path = "/content/dummy_medical_dataset.json"  # <-- make sure your uploaded file name matches this

# Read and parse JSON file
with open(file_path, "r") as f:
    data = json.load(f)

# Extract "dataset" entries into a DataFrame
records = data["dataset"]
df = pd.json_normalize(records)

# Display confirmation and preview
print("✅ JSON file loaded successfully!")
print(f"Total records: {len(df)}\n")
print("Columns available:", list(df.columns), "\n")

# Show sample of important columns
print(df[[
    "question",
    "ground_truth_context",
    "ground_truth_answer",
    "api_response_details.search_summarization"
]].head())


✅ JSON file loaded successfully!
Total records: 10

Columns available: ['question', 'ground_truth_context', 'ground_truth_answer', 'diagnosis_id', 'diagnosis_title', 'difficulty_level', 'category', 'source_file', 'api_response_details.search_summarization', 'api_response_details.chat_summarization'] 

                                            question  \
0  What is the background and epidemiology of chr...   
1  What tests are recommended for evaluating chro...   
2  What is the management approach for chronic st...   
3  What is the background and epidemiology of acr...   
4                       How is acromegaly diagnosed?   

                                ground_truth_context  \
0  [Chronic stable angina is caused by myocardial...   
1  [Diagnosis is based on history, ECG during pai...   
2  [Management includes lifestyle changes, antipl...   
3  [Acromegaly is caused by excessive growth horm...   
4  [Diagnosis is confirmed by elevated IGF-1 and ...   

                       

##precision

In [ ]:
def precision(reference, retrieved_chunks, query=None):
    """
    Precision evaluator prompt for medical-domain datasets.
    Returns a formatted prompt string for GPT-4o evaluation.
    """

    prompt_template = f"""
**Role**
You are an expert evaluator tasked with assessing the relevance of a retrieved context chunk for a given query using the reference answer.
Your goal is to produce a detailed traceback table and compute a Context Precision score.

**Information to Use**
Original Prompt: {query}
Reference Answer: {reference}
Retrieved Chunks (with metadata): {retrieved_chunks}

---

**Task**
1. Compare each retrieved chunk with the reference answer using semantic relevance.
2. For each chunk, determine:
- **Relevance**: Yes → if it contains information semantically aligned with the reference, No → otherwise.
- **Rank**: 1
   - **Contribution**: Rank 1 relevant → 0.99, not relevant → 0.0
   - **Failure Condition**: e.g.,
      - **Hallucination** → fabricated or medically incorrect content not present in the reference
      - **Off-topic** → unrelated or irrelevant to the query or reference theme
      - **Contradiction** → states the opposite or conflicts with the reference
      - **Partial relevance** → contains some correct information but misses key elements
      - **Generic / Vague** → overly broad statements lacking specific factual alignment
      - **Context drift** → discusses adjacent topics but not the exact question focus
      - **Incomplete** → stops early or omits essential answer details
      - **Ambiguous** → unclear phrasing or uncertain factual meaning
      - **Repetition** → repeats the query or reference without adding factual support
      - **Noise / Placeholder** → meaningless or placeholder text unrelated to content
   - **Explanation**: short justification for why the chunk was or was not relevant.

3. Generate a **Traceback Table** with columns:
   | Rank | Retrieved Chunk | Relevant? | Contribution | Failure Condition | Explanation |

4. Compute the **Final Context Precision Score (0 – 1)**:
If relevant → 0.99, else 0.0

---

### 🔹 Example 1 – Relevant (Context Precision = 0.99)
**Reference:**
"Stable angina occurs due to fixed narrowing of coronary arteries, causing predictable chest pain with exertion."

**Retrieved Chunk:**
"Stable angina is caused by reduced blood flow through narrowed coronary arteries, typically triggered by physical activity."

**Traceback Table:**
| Rank | Retrieved Chunk | Relevant? | Contribution | Failure Condition | Explanation |
|------|-----------------|-----------|--------------|------------------|-------------|
| 1 | Stable angina is caused by reduced blood flow through narrowed coronary arteries… | Yes | 0.99 | None | Accurately matches the reference mechanism and context. |

**Final Precision Score:** 0.99

---

### 🔹 Example 2 – Not Relevant (Context Precision = 0.0)
**Reference:**
"Stable angina occurs due to fixed narrowing of coronary arteries, causing predictable chest pain with exertion."

**Retrieved Chunk:**
"Angina is a gastrointestinal disorder that affects the stomach lining and causes indigestion."

**Traceback Table:**
| Rank | Retrieved Chunk | Relevant? | Contribution | Failure Condition | Explanation |
|------|-----------------|-----------|--------------|------------------|-------------|
| 1 | Angina is a gastrointestinal disorder that affects the stomach lining and causes indigestion. | No | 0.0 | Hallucination | Completely unrelated to the cardiac cause of angina. |

**Final Precision Score:** 0.0


---
**Note:**
In this dataset, each query has only **one retrieved context**.
Therefore, the Context Precision metric is **binary**:
- 1 (≈ 0.99) → retrieved context supports the reference answer
- 0 → retrieved context does not support the reference answer
The traceback table simply documents the justification for each decision.

**Guidelines**
- Evaluate semantic overlap only — not surface wording.
- Base judgments **strictly** on the provided reference and chunks.
- Avoid outside medical facts or stylistic feedback.
- Return output **strictly in JSON**:
{{
  "Score": <float>,
  "Traceback": "<table or JSON array>"
}}
"""
    return prompt_template


In [ ]:
import pandas as pd
import json
from tqdm import tqdm

# Initialize storage for results
precision_scores = []
tracebacks = []
# Always reload fresh dataset before running a metric
with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])



print("🧠 Starting precision evaluation using GPT-4o...")
print(f"Total records to process: {len(df)}\n")

# Loop through dataset rows
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        query = row.get("question", "")
        retrieved_chunks = row.get("ground_truth_context", "")
        reference = row.get("api_response_details.search_summarization", "")

        # Build precision evaluation prompt for this row
        prompt = precision(reference=reference, retrieved_chunks=retrieved_chunks, query=query)

        # Send to Azure OpenAI (GPT-4o)
        response = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "Return only valid JSON with keys 'Score' and 'Traceback'. No markdown or text outside JSON."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=2500
        )

        # Extract raw model output
        content = response.choices[0].message.content.strip()

        # Remove any stray characters and parse as JSON
        content = content.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(content)

        precision_scores.append(parsed.get("Score", None))
        tracebacks.append(parsed.get("Traceback", None))

    except Exception as e:
        precision_scores.append(None)
        tracebacks.append(f"Error: {str(e)}")

# Add results to DataFrame
df["Precision_Score"] = precision_scores
df["Precision_Traceback"] = tracebacks

# Save to Excel
output_file = "/content/evaluated_precision_results.xlsx"
df.to_excel(output_file, index=False)

print("\n✅ Precision evaluation complete!")
print(f"📂 Results saved to: {output_file}")

# Display preview of important columns
print(df[["question", "Precision_Score", "Precision_Traceback"]].head())


🧠 Starting precision evaluation using GPT-4o...
Total records to process: 10



100%|██████████| 10/10 [00:19<00:00,  1.93s/it]


✅ Precision evaluation complete!
📂 Results saved to: /content/evaluated_precision_results.xlsx
                                            question  Precision_Score  \
0  What is the background and epidemiology of chr...             0.99   
1  What tests are recommended for evaluating chro...             0.99   
2  What is the management approach for chronic st...             0.99   
3  What is the background and epidemiology of acr...             0.99   
4                       How is acromegaly diagnosed?             0.99   

                                 Precision_Traceback  
0  [{'Rank': 1, 'Retrieved Chunk': 'Chronic stabl...  
1  [{'Rank': 1, 'Retrieved Chunk': 'Diagnosis is ...  
2  [{'Rank': 1, 'Retrieved Chunk': 'Management in...  
3  [{'Rank': 1, 'Retrieved Chunk': 'Acromegaly is...  
4  [{'Rank': 1, 'Retrieved Chunk': 'Diagnosis is ...  


##Recall

In [ ]:
def recall_prompt(INSERT_QUERY, INSERT_REFERENCE, INSERT_RETRIEVED_CONTEXTS):
    """
    Context Recall evaluator prompt
    Measures how well the retrieved contexts cover the reference answer facts.
    """

    diagnostic_prompt = f"""
You are an expert evaluator. Your task is to compute **Context Recall**.

**Definition:**
- Context Recall = fraction of **reference facts** that are present in the retrieved context chunks.

**Instructions:**
1. Break the reference answer into individual factual statements or claims.
2. For each fact, check if it is **found** (semantically) in at least one retrieved context chunk.
3. Create a table with two columns:
   - Reference Fact
   - Found in Retrieved Contexts? (Valid/Invalid)
4. Compute Recall = (# of valid facts) ÷ (total number of reference facts)
5. Output must include:
   - The Recall Table
   - The Final Recall Score (0–1)

---

### 🔹 Few-Shot Example

user_input: "What is the background and epidemiology of chronic stable angina?"
reference: "It occurs due to fixed coronary artery narrowing causing exertional chest pain that resolves with rest."
retrieved_contexts: [
  "Chronic stable angina occurs when fixed atherosclerotic narrowing of the coronary arteries causes exertional chest pain that subsides with rest or nitrates."
]

**Expected Output**
Recall Table:
| Reference Fact                                       | Found?  |
|------------------------------------------------------|---------|
| Fixed coronary artery narrowing causes angina        | Valid   |
| Pain occurs during exertion                          | Valid   |
| Pain resolves with rest                              | Valid   |
Recall = 3 / 3 = 1.0

---

Now evaluate:
user_input: "{INSERT_QUERY}"
reference: "{INSERT_REFERENCE}"
retrieved_contexts: {INSERT_RETRIEVED_CONTEXTS}
"""

    return diagnostic_prompt


In [ ]:
import json
from tqdm import tqdm

# Initialize storage for recall results
recall_scores = []
recall_tables = []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])


print("🧠 Starting Recall evaluation using GPT-4o...")
print(f"Total records to process: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        INSERT_QUERY = row.get("question", "")
        INSERT_RETRIEVED_CONTEXTS = row.get("ground_truth_context", "")
        # Recall uses Ground Truth Answer (NOT search_summarization)
        INSERT_REFERENCE = row.get("ground_truth_answer", "")

        # Build recall evaluation prompt
        recall_prompt_text = recall_prompt(
            INSERT_QUERY=INSERT_QUERY,
            INSERT_REFERENCE=INSERT_REFERENCE,
            INSERT_RETRIEVED_CONTEXTS=INSERT_RETRIEVED_CONTEXTS
        )

        # Send to Azure OpenAI (GPT-4o)
        response = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {
                    "role": "system",
                    "content": "Return only valid JSON with keys 'Recall_Score' and 'Recall_Table'. No markdown or text outside JSON."
                },
                {"role": "user", "content": recall_prompt_text}
            ],
            temperature=0.0,
            max_tokens=2500
        )

        # Extract and clean the output
        content = response.choices[0].message.content.strip()
        content = content.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(content)

        recall_scores.append(parsed.get("Recall_Score", None))
        recall_tables.append(parsed.get("Recall_Table", None))

    except Exception as e:
        recall_scores.append(None)
        recall_tables.append(f"Error: {str(e)}")

# Add Recall columns to DataFrame
df["Recall_Score"] = recall_scores
df["Recall_Table"] = recall_tables

# Save Recall results separately
output_file = "/content/evaluated_recall_results.xlsx"
df.to_excel(output_file, index=False)

print("\n✅ Recall evaluation complete!")
print(f"📂 Results saved to: {output_file}")

# Display preview
print(df[["question", "Recall_Score"]].head())


🧠 Starting Recall evaluation using GPT-4o...
Total records to process: 10



100%|██████████| 10/10 [00:15<00:00,  1.50s/it]


✅ Recall evaluation complete!
📂 Results saved to: /content/evaluated_recall_results.xlsx
                                            question  Recall_Score
0  What is the background and epidemiology of chr...           1.0
1  What tests are recommended for evaluating chro...           1.0
2  What is the management approach for chronic st...           0.8
3  What is the background and epidemiology of acr...           1.0
4                       How is acromegaly diagnosed?           1.0


##hallucination_cove

In [ ]:
def hallucination_cove(answer: str):
    """
    Checks consistency of a model-generated answer using verification questions and comparison.
    Uses Azure GPT-4o client.
    """

    initial_answer = answer.strip()
    if not initial_answer:
        return None

    print("Initial answer:", initial_answer)

    # === Step 1: Generate verification questions ===
    generate_prompt = f"""
You are a medical information verification expert.

Your task is to generate exactly **2 verification questions** that help check the **factual accuracy** and **medical validity** of the given answer.

---

### PURPOSE
These verification questions will be used to detect **hallucinations**, **factual errors**, or **loss of meaning** in medical explanations.
They must focus on confirming key facts within the same context as the given answer.

---

### INSTRUCTIONS

1. **Stay strictly within the medical scope of the answer.**
   - Do NOT introduce new conditions, mechanisms, or treatments not mentioned in the answer.
   - Do NOT speculate or expand beyond the given information.
   - Use only factual aspects that can be verified within the same medical topic.

2. **Focus on medically relevant verification.**
   Each question should confirm or probe one important factual point, such as:
   - Pathophysiology or underlying mechanism
   - Symptom or clinical feature
   - Diagnostic or epidemiological aspect
   - Management principle (only if discussed)

3. **Avoid non-factual or broad questions.**
   - No open-ended prompts like “Explain more…” or “What are other causes?”
   - Ask focused verification questions that confirm medical facts present in the answer.

4. **Formatting Rules**
   - Write exactly **2 short, clear questions**, one per line.
   - Use professional, neutral medical language.
   - Do **not** add numbering, labels, or commentary.
   - Output only the question text — no extra sentences.

---

### EXAMPLES

**Example 1**
Answer: "Stable angina occurs due to fixed narrowing of the coronary arteries, causing predictable chest pain during exertion."
Output:
What causes stable angina?
When does chest pain typically occur in stable angina?

**Example 2**
Answer: "Hypothyroidism results from decreased production of thyroid hormones, leading to fatigue, weight gain, and cold intolerance."
Output:
What is the primary physiological cause of hypothyroidism?
What are common symptoms seen in hypothyroidism?

**Example 3**
Answer: "Type 2 diabetes mellitus occurs due to insulin resistance and relative insulin deficiency, commonly associated with obesity and metabolic syndrome."
Output:
What mechanisms contribute to Type 2 diabetes mellitus?
What factors are commonly associated with Type 2 diabetes?

---

### INPUT
Answer:
{initial_answer}

---

Now generate exactly 2 verification questions that check factual correctness of the above medical answer.
"""


    verification_response = client.chat.completions.create(
        model=GPT4O_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": "Return only 2 plain-text questions, one per line."},
            {"role": "user", "content": generate_prompt}
        ],
        temperature=0
    )
    verification_questions = verification_response.choices[0].message.content.strip().split("\n")
    print("\nVerification Questions:", verification_questions)

    # === Step 2: Generate verification answers ===
    verification_answers = []
    for q in verification_questions:
        ans_response = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[{"role": "user", "content": q}],
            temperature=0
        )
        verification_answers.append(ans_response.choices[0].message.content.strip())

    print("\nVerification Answers:", verification_answers)

    # === Step 3: Evaluate consistency ===
    eval_prompt = f"""
You are a **medical information verification expert**.

Your task is to assess whether the **verification answers** accurately reflect and support the **original medical answer**.
You must identify any hallucination, factual deviation, or loss of meaning between the answers.

---

### INPUT DATA

**Original Answer:**
{initial_answer}

**Verification Answers:**
1. {verification_answers[0]}
2. {verification_answers[1]}

---

### TASK OBJECTIVE

Compare the verification answers to the original medical answer and determine:

- Do they express the **same factual meaning** and **clinical interpretation**?
- Have they **introduced**, **omitted**, or **altered** any medical facts?
- Are there any signs of **hallucination**, **fabrication**, or **misrepresentation** of the medical content?

Your evaluation should focus on factual accuracy and semantic alignment — not stylistic similarity.

---

### DRIFT TYPE DEFINITIONS

- **Fully Aligned:**
  Verification answers correctly restate or paraphrase the same medical facts and concepts as the original answer.
  (Example: “Caused by coronary narrowing” ↔ “Due to fixed narrowing of the coronary arteries.”)

- **Minor Deviation:**
  Verification answers simplify or omit small details but still preserve the same overall medical meaning.
  (Example: “Atherosclerotic narrowing” → “narrowed arteries.”)

- **Factual Deviation:**
  Verification answers introduce, alter, or remove important details that change the medical meaning or accuracy.
  (Example: “Caused by infection” instead of “caused by ischemia.”)

- **Contradiction:**
  Verification answers directly oppose or negate a key element of the original answer.
  (Example: “Occurs at rest” vs “Occurs during exertion.”)

---

### 🔹 SCORING CRITERIA

Assign a **factual_alignment_score** (float between 0.0 and 1.0):

| Score | Interpretation |
|--------|----------------|
| 1.00 | Fully Aligned — no hallucination or deviation |
| 0.75 | Minor Deviation — meaning preserved overall |
| 0.50 | Moderate Factual Deviation — partial change in meaning |
| 0.25 | Significant deviation or partial contradiction |
| 0.00 | Direct contradiction or full hallucination |

---

### OUTPUT FORMAT

Return your evaluation **strictly in valid JSON format**.
Do not include markdown, commentary, or extra text.

**Example JSON:**
{{
  "factual_alignment_score": 0.50,
  "drift_type": "Factual Deviation",
  "reasoning": "Verification answers introduced a different cause of disease, changing the factual interpretation."
}}

---

Now analyze the provided medical answers and return your final judgment in JSON format only.
"""


    eval_response = client.chat.completions.create(
        model=GPT4O_DEPLOYMENT_NAME,
        messages=[
            {"role": "system", "content": "Return valid JSON, no markdown or commentary."},
            {"role": "user", "content": eval_prompt}
        ],
        temperature=0
    )

    result = eval_response.choices[0].message.content.strip()
    print("\n🧩 Evaluation JSON:\n", result)
    return result


In [ ]:
# ====== Cell 5: Evaluate Factual Alignment (Hallucination Detection) for All Rows ======
import json
from tqdm import tqdm

# Initialize result storage
alignment_scores, drift_types, reasoning_list, raw_json = [], [], [], []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🧠 Starting hallucination / factual alignment evaluation using GPT-4o...")
print(f"Total records to process: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        answer = row.get("api_response_details.search_summarization", "")
        if not answer or pd.isna(answer):
            alignment_scores.append(None)
            drift_types.append(None)
            reasoning_list.append(None)
            raw_json.append(None)
            continue

        # Run the CoVe hallucination / factual drift evaluator
        result = hallucination_cove(answer)

        # Parse JSON output safely
        try:
            parsed = json.loads(result)
        except Exception:
            print(f"⚠️ Row {i}: Invalid JSON output. Saving raw text.")
            parsed = {"factual_alignment_score": None, "drift_type": "ParseError", "reasoning": str(result)}

        alignment_scores.append(parsed.get("factual_alignment_score"))
        drift_types.append(parsed.get("drift_type"))
        reasoning_list.append(parsed.get("reasoning"))
        raw_json.append(parsed)

    except Exception as e:
        alignment_scores.append(None)
        drift_types.append(None)
        reasoning_list.append(f"Error: {e}")
        raw_json.append(None)

# Store results in DataFrame
df["Factual_Alignment_Score"] = alignment_scores
df["Drift_Type"] = drift_types
df["Alignment_Reasoning"] = reasoning_list
df["Alignment_JSON"] = raw_json

# ====== Save Results Separately ======
output_path = "/content/evaluated_hallucination_cove.xlsx"
df.to_excel(output_path, index=False)

print("\n✅ Factual alignment (hallucination) results complete!")
print(f"📂 Saved to: {output_path}")

# Display summary preview
print(df[["question", "Factual_Alignment_Score", "Drift_Type"]].head())


🧠 Starting hallucination / factual alignment evaluation using GPT-4o...
Total records to process: 10



  0%|          | 0/10 [00:00<?, ?it/s]

Initial answer: Chronic stable angina occurs when fixed atherosclerotic narrowing of the coronary arteries causes exertional chest pain that subsides with rest or nitrates.

Verification Questions: ['What is the underlying cause of chronic stable angina?  ', 'How is chest pain in chronic stable angina typically relieved?']

Verification Answers: ["The underlying cause of **chronic stable angina** is typically **myocardial ischemia**, which occurs when the heart muscle does not receive enough oxygen-rich blood to meet its demands. This is most commonly due to **atherosclerosis**—the buildup of fatty plaques in the coronary arteries that narrows the vessels and reduces blood flow. \n\nKey factors contributing to chronic stable angina include:\n\n1. **Coronary artery disease (CAD):** The narrowing of coronary arteries due to plaque buildup limits blood flow, especially during physical exertion or stress when the heart's oxygen demand increases.\n   \n2. **Imbalance between oxygen supply a

 10%|█         | 1/10 [00:09<01:23,  9.32s/it]


🧩 Evaluation JSON:
 {
  "factual_alignment_score": 1.00,
  "drift_type": "Fully Aligned",
  "reasoning": "The verification answers accurately restate and expand upon the original medical answer without introducing any factual inaccuracies or altering the clinical interpretation. They provide additional context, such as the role of endothelial dysfunction and the mechanism of nitroglycerin, but these details align with the original explanation of chronic stable angina being caused by fixed atherosclerotic narrowing of the coronary arteries, leading to exertional chest pain relieved by rest or nitrates."
}
Initial answer: Evaluation of chronic stable angina involves ECG recording, exercise stress tests, and coronary angiography in selected cases.

Verification Questions: ['What diagnostic tests are commonly used to evaluate chronic stable angina?  ', 'In which cases is coronary angiography typically performed for chronic stable angina?']

Verification Answers: ["Chronic stable angina is

...


🧩 Evaluation JSON:
 {
  "factual_alignment_score": 0.25,
  "drift_type": "Significant Deviation",
  "reasoning": "The verification answers contradict the original answer by stating that chest CT and brain MRI are not commonly used to evaluate arrhythmias or clot formation in the context of arrhythmias. Instead, they provide alternative diagnostic tools and imaging modalities, which significantly alters the original medical interpretation. While the verification answers are factually accurate in their own right, they do not align with the original answer's claims and introduce a different clinical perspective."
}
Initial answer: Arrhythmia can be managed with antibiotics and pain relievers.

Verification Questions: ['Can arrhythmia be treated with antibiotics?  ', 'Are pain relievers a standard management option for arrhythmia?']

Verification Answers: ["No, arrhythmia cannot be treated with antibiotics. Arrhythmia refers to an irregular heartbeat, which is typically caused by issues w

 90%|█████████ | 9/10 [01:52<00:11, 11.44s/it]


🧩 Evaluation JSON:
 {
  "factual_alignment_score": 0.00,
  "drift_type": "Contradiction",
  "reasoning": "The verification answers directly contradict the original answer. The original answer states that arrhythmia can be managed with antibiotics and pain relievers, which is factually incorrect. The verification answers correctly clarify that antibiotics and pain relievers are not appropriate treatments for arrhythmia, providing accurate alternative management options. This creates a direct opposition to the original answer's claims."
}
Initial answer: Cardiology referral is rarely required for arrhythmia unless hypertension is severe.

Verification Questions: ['When is cardiology referral typically indicated for arrhythmia?  ', 'Does severe hypertension commonly necessitate cardiology referral for arrhythmia?']

Verification Answers: ['A cardiology referral is typically indicated for arrhythmias in the following situations:\n\n### 1. **Symptomatic Arrhythmias**\n   - **Palpitations**

100%|██████████| 10/10 [02:09<00:00, 12.94s/it]


🧩 Evaluation JSON:
 {
  "factual_alignment_score": 0.50,
  "drift_type": "Moderate Factual Deviation",
  "reasoning": "The verification answers expand significantly on the original answer by introducing multiple scenarios and conditions where cardiology referral for arrhythmias may be warranted. While some of these points align with the original answer's implication that severe hypertension can contribute to arrhythmias requiring referral, the verification answers introduce additional contexts (e.g., structural heart disease, genetic syndromes, special populations) that were not mentioned in the original answer. This alters the scope and meaning of the original statement, which focused on the rarity of cardiology referral for arrhythmias unless hypertension is severe."
}

✅ Factual alignment (hallucination) results complete!
📂 Saved to: /content/evaluated_hallucination_cove.xlsx
                                            question  Factual_Alignment_Score  \
0  What is the background 

##hallucination_consistency

In [ ]:
def hallucination_consistent(question, context):
    """
    Builds a detailed evaluation prompt that tests response consistency
    by generating 3 answers from the same context, then evaluating for factual or semantic drift.
    """

    prompt_template = f"""
You are an expert medical information evaluator, trained to detect internal inconsistencies, factual drift, or hallucinations
within generated medical answers.

Your task is to perform a structured two-phase analysis using only the provided medical context.

---

### 1️⃣ Generation Phase
- Generate **three distinct answers** to the given question.
- Use **only** the provided context as your factual source.
- Avoid introducing assumptions or external medical information.
- Minor wording differences are acceptable, but the **factual meaning** must remain identical.

---

### 2️⃣ Evaluation Phase
- Compare all three generated answers.
- Assess whether they maintain **semantic and factual consistency**.
- Identify any of the following types of drift:
  - **Contradiction:** One answer directly conflicts with another.
  - **Factual Drift:** One answer introduces or removes key medical facts.
  - **Informational Drift:** One answer adds unnecessary, ungrounded, or missing details.

If all three answers remain semantically and factually aligned, mark them as consistent.
Otherwise, describe the type of drift and explain why.

---

### 🔹 Output Format (Strict JSON)
{{
  "generated_answers": {{
    "answer_1": "First context-grounded answer",
    "answer_2": "Second context-grounded answer",
    "answer_3": "Third context-grounded answer"
  }},
  "evaluation": {{
    "consistency_score": "float 0.0–1.0 (0.0 = severe drift, 1.0 = perfect consistency)",
    "is_consistent": "true/false",
    "drift_type": "No Drift | Contradiction | Factual Drift | Informational Drift",
    "reasoning": "Concise explanation supporting the decision",
    "problematic_statements": [
      {{
        "answer_1_statement": "Conflicting or drifted statement from answer 1",
        "answer_2_statement": "Conflicting or drifted statement from answer 2",
        "answer_3_statement": "Conflicting or drifted statement from answer 3 (if applicable)"
      }}
    ]
  }}
}}

---

Now perform this analysis based on the provided input.

**Context:**
{context}

**Question:**
{question}

Respond strictly in **valid JSON** format.
"""
    return prompt_template


In [ ]:
# ====== Cell: Evaluate Consistency Across Generated Answers ======
import json
from tqdm import tqdm

# Initialize lists to store results
consistency_scores, drift_types, reasoning_list, raw_json = [], [], [], []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🧠 Starting consistency evaluation using GPT-4o...")
print(f"Total records to process: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        # Extract the question and its supporting context
        question = row.get("question", "")
        context = row.get("ground_truth_context", "")

        # Skip if either field is empty
        if not question or pd.isna(question) or not context or pd.isna(context):
            consistency_scores.append(None)
            drift_types.append(None)
            reasoning_list.append(None)
            raw_json.append(None)
            continue

        # Build the consistency evaluation prompt
        consistency_prompt = hallucination_consistent(question, context)

        # Send to Azure GPT-4o for evaluation
        response = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {
                    "role": "system",
                    "content": "Return only valid JSON with keys 'generated_answers' and 'evaluation'. No markdown or commentary."
                },
                {"role": "user", "content": consistency_prompt}
            ],
            temperature=0.0,
            max_tokens=3000
        )

        # Extract the model response
        result_text = response.choices[0].message.content.strip()
        result_text = result_text.replace("```json", "").replace("```", "").strip()

        # Parse JSON safely
        try:
            parsed = json.loads(result_text)
            eval_data = parsed.get("evaluation", {})
        except Exception:
            print(f"⚠️ Row {i}: Invalid JSON output. Saving raw text.")
            eval_data = {"consistency_score": None, "drift_type": "ParseError", "reasoning": result_text}

        # Store results
        consistency_scores.append(eval_data.get("consistency_score"))
        drift_types.append(eval_data.get("drift_type"))
        reasoning_list.append(eval_data.get("reasoning"))
        raw_json.append(parsed)

    except Exception as e:
        consistency_scores.append(None)
        drift_types.append(None)
        reasoning_list.append(f"Error: {e}")
        raw_json.append(None)

# Add new columns to the DataFrame
df["Consistency_Score"] = consistency_scores
df["Drift_Type"] = drift_types
df["Consistency_Reasoning"] = reasoning_list
df["Consistency_JSON"] = raw_json

# ====== Save Results to Excel ======
output_path = "/content/evaluated_hallucination_consistent.xlsx"
df.to_excel(output_path, index=False)

print("\n✅ Consistency evaluation complete!")
print(f"📂 Results saved to: {output_path}")

# Display summary preview
print(df[["question", "Consistency_Score", "Drift_Type"]].head())


🧠 Starting consistency evaluation using GPT-4o...
Total records to process: 10



100%|██████████| 10/10 [00:29<00:00,  2.91s/it]


✅ Consistency evaluation complete!
📂 Results saved to: /content/evaluated_hallucination_consistent.xlsx
                                            question  Consistency_Score  \
0  What is the background and epidemiology of chr...                1.0   
1  What tests are recommended for evaluating chro...                1.0   
2  What is the management approach for chronic st...                1.0   
3  What is the background and epidemiology of acr...                1.0   
4                       How is acromegaly diagnosed?                1.0   

  Drift_Type  
0   No Drift  
1   No Drift  
2   No Drift  
3   No Drift  
4   No Drift  


##multiturn_relevance

In [ ]:
# ====== Chat Relevance Prompt Builder (Chat_Relevance_V2) ======
def chat_relevance_prompt(question, response, chat_history="[]", persona_details="None"):
    """
    Builds a full Chat_Relevance_V2 evaluation prompt.
    Supports optional chat history and persona.
    """
    template = f"""
You are an expert evaluator tasked with determining the relevance of a given LLM response to a user input,
considering the preceding chat history and, if applicable, a defined persona. While calculating the score,
please check the user input and LLM Response, giving higher priority to the immediate question and answer
from the Chat History. Before proceeding, read the entire Chat History.

**User Input asked by the user to chatbot:** {question}

**LLM Response for evaluation from chatbot:** {response}

**Chat History:** {chat_history}
If Chat History is empty, then the question is asked in a new chat session, so chat history dependency is not required.

**Defined Persona (If Applicable):** {persona_details}
-------------------------------
Chat History structure example:
<chat_history>
     [
        Dict_0:<Role:"user", <content:{{user_question_1 about AI}}>,
        Dict_1:<Role:"assistant",<content:{{language_model_response about Artificial Intelligence}}>,
        Dict_2:<Role:"user", <content:{{user_question_2 please rewrite/give some more info}}>,
        Dict_3:<Role:"assistant",<content:{{language_model_response should focused on Artificial Intelligence}}>,
        Dict_4:<Role:"user", <content:{{user_question_3 What is financial Stability}}>,
        Dict_5:<Role:"assistant",<content:{{language_model_response should address financial Stability}}>(High Priority for current question),
        so on.....
    ]
</chat_history>

For example:
Correct scenario based on Dict_5 from <chat_history>:
1. If User Input asked to rewrite/give additional info then LLM Answer should address about financial Stability.
2. If User Input asked about rules and regulation of some country then LLM Answer should have no connection between previous conversation so LLM Answer should be about rules and regulation of that country.

Wrong scenario based on Dict_5 from <chat_history>:
1. If User Input asked about to "rewrite again the info", If LLM Answer address about part of Artificial Intelligence then it is abruptly wrong. It is a failure scenario and reduce the score give the detailed reason like "the LLM Answer doesn't adhered with the previous conversation....."

Keep this as reference example while checking the Conflicting Information and Consideration of Chat History criteria.

**Evaluation Criteria:**
1. **Direct Topical Relevance (Weight: 1.0):** How directly and accurately does the LLM's response address the core question, request, or statement in the "User Input," independent of any persona? Is the response on the same subject? Does it provide a clear and relevant answer or action?
2. **Persona Consistency (Weight: 1.5 - Applicable ONLY if a Persona is Defined):** If a "Defined Persona" is provided, how consistently and effectively does the LLM's response embody that persona in terms of tone, style, language, behaviours, and any other defined characteristics? Are there any noticeable deviations from the established persona? (If no persona is defined, this criterion receives a score of N/A and its weight is not factored into the total).
3. **Absence of Conflicting Information (Weight: 1.5):** Does the current response contradict any information, agreements, or established context within the "Chat History" or any aspects of the "Defined Persona" (if applicable)?
4. **Efficiency and Conciseness (within Relevance) (Weight: 0.5):** Is the response reasonably concise and focused on the relevant aspects of the conversation and persona (if applicable), avoiding unnecessary information, tangents, or overly verbose language?
5. **Consideration of Chat History (Weight: 3.5):**
   * If the user expressed dissatisfaction in previous turns, explain whether and how the current response attempts to rectify that. Quote the relevant turns.
   * If the user explicitly asked for a rephrasing or a new idea in previous turns, explain whether and how the current response fulfills that specific request. Quote the relevant turns.
   * Identify any instances where the current response could have better built upon previously provided information or adapted more effectively to the evolving user intent throughout the conversation. Explain what the LLM could have done differently.

**Scoring Guidance (0-10):**

* **9-10 - Exceptionally Relevant:** The response is perfectly aligned topically with the current input and chat history. If a persona is defined, it is also flawlessly and consistently embodied.
* **7-8 - Highly Relevant:** The response is strongly relevant topically and demonstrates good awareness of the chat history. If a persona is defined, it is consistently maintained with only minor imperfections.
* **5-6 - Moderately Relevant:** The response addresses the main points of the user input and shows some connection to the chat history. If a persona is defined, it is present but may have some noticeable inconsistencies or areas for improvement.
* **3-4 - Poorly Relevant:** The response only partially addresses the user input or shows a weak connection to the chat history. If a persona is defined, there are significant deviations from the established character.
* **1-2 - Very Poorly Relevant:** The response is largely irrelevant to the user input and/or chat history. If a persona is defined, it is poorly or inconsistently applied.
* **0 - Not Relevant:** The response fails to address the user input or is completely disconnected from the conversation history. If a persona is defined, it is either entirely absent or actively contradicted.

**Important Instruction:**
The current LLM response might attempt to be more helpful or convenient for the user by slightly altering the current response based on very old chat conversation. **You must be strict in assigning the score and penalize any deviation from the immediate question and answer from the chat_history, even if the intention seems helpful.** The goal is to evaluate the LLM's ability to consistently embody the chat_history, not to prioritize user convenience over response accuracy.

Evaluate the relevance of the LLM's response based on its topical connection to the user's current input and the preceding chat history.

* **If a 'Defined Persona' is provided:** Also meticulously assess how well the response embodies that persona across all defined characteristics. The final relevance score should reflect a holistic evaluation of both topical relevance and persona consistency. A response that is strongly in character but fails to adequately address the user's needs within the conversational context is not truly relevant.
* **If No Persona Defined is indicated:** Focus solely on the topical relevance of the response to the user input and the chat history. Persona consistency should not be considered in your evaluation.

Provide a detailed justification for your assigned score, referencing specific parts of the "User Input," "LLM Response," and "Chat History." If a persona is defined, explicitly address its consistency within your justification. Explain how you weighed the different criteria in arriving at your final score.

Give the output in JSON FORMAT only as mentioned below because I will apply json.loads on the response you are giving.
So don't add any delimiter:
score: <The score 0-10 based on the given criteria (lower is worse)>
reason: <detailed explanation of the reasoning behind the assigned score, referencing specific parts of the input, response, and chat history, and the defined persona (if applicable). Clearly articulate how both topical relevance and persona consistency (when applicable) influenced your score, considering the assigned weights. Provide it in string format>
"""
    return template.strip()

In [ ]:
# ====== Chat Relevance Evaluation (Multi-turn with optional history) ======
import json
from tqdm import tqdm
import pandas as pd

# Optional: build simple simulated chat history using context fields
def build_chat_history(row):
    """
    Builds a small simulated chat history using diagnosis title, context, and current question.
    If your dataset already contains real chat turns, you can directly use them instead.
    """
    diag = row.get("diagnosis_title", "")
    ctx_list = row.get("ground_truth_context", [])
    context = ctx_list[0] if isinstance(ctx_list, list) and len(ctx_list) > 0 else ""
    prev_qa = []
    if diag:
        prev_qa.append({"role": "user", "content": f"What is {diag.lower()}?"})
        prev_qa.append({"role": "assistant", "content": context})
    current_q = row.get("question", "")
    if current_q:
        prev_qa.append({"role": "user", "content": current_q})
    return json.dumps(prev_qa, ensure_ascii=False)

# Evaluate each row
chat_relevance_scores = []
chat_relevance_reasons = []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🧠 Starting Chat Relevance evaluation (multi-turn mode)...")
print(f"Total records: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        question = row.get("question", "")
        response = row.get("api_response_details.chat_summarization", "")
        if not question or not response:
            chat_relevance_scores.append(None)
            chat_relevance_reasons.append("Missing question or response")
            continue

        chat_history = build_chat_history(row)
        persona_details = row.get("persona", "None")

        # Build prompt
        prompt = chat_relevance_prompt(
            question=question,
            response=response,
            chat_history=chat_history,
            persona_details=persona_details
        )

        # Send to Azure GPT-4o
        resp = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "Return only valid JSON with keys 'score' and 'reason'. No markdown or extra text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=900
        )

        raw = resp.choices[0].message.content.strip()
        cleaned = raw.replace("```json", "").replace("```", "").strip()

        # Safe JSON parse
        try:
            parsed = json.loads(cleaned)
            chat_relevance_scores.append(parsed.get("score"))
            chat_relevance_reasons.append(parsed.get("reason"))
        except Exception:
            chat_relevance_scores.append(None)
            chat_relevance_reasons.append(f"Invalid JSON: {cleaned}")

    except Exception as e:
        chat_relevance_scores.append(None)
        chat_relevance_reasons.append(f"Error: {e}")

# Append results to DataFrame
df["Chat_Relevance_Score_MultiTurn"] = chat_relevance_scores
df["Chat_Relevance_Reason_MultiTurn"] = chat_relevance_reasons

# Save updated results
output_file = "/content/evaluated_chat_relevance_multiturn.xlsx"
df.to_excel(output_file, index=False)

print("\n✅ Chat Relevance evaluation complete (multi-turn mode)!")
print(f"📂 Results saved to: {output_file}")
print(df[["question", "Chat_Relevance_Score_MultiTurn"]].head())


🧠 Starting Chat Relevance evaluation (multi-turn mode)...
Total records: 10



100%|██████████| 10/10 [00:36<00:00,  3.63s/it]


✅ Chat Relevance evaluation complete (multi-turn mode)!
📂 Results saved to: /content/evaluated_chat_relevance_multiturn.xlsx
                                            question  \
0  What is the background and epidemiology of chr...   
1  What tests are recommended for evaluating chro...   
2  What is the management approach for chronic st...   
3  What is the background and epidemiology of acr...   
4                       How is acromegaly diagnosed?   

   Chat_Relevance_Score_MultiTurn  
0                               3  
1                               9  
2                               8  
3                               3  
4                               6  


##qa_relevance

In [ ]:
def qa_relevance(question, response):
    prompt_template = f"""
You are a RELEVANCE grader; providing the relevance of the given RESPONSE to the given QUESTION.

Respond only as a number from 0 to 10 where 0 is the least relevant and 10 is the most relevant.

A few additional scoring guidelines:
- Long RESPONSES should score equally well as short RESPONSES.
- Answers that intentionally do not answer the question, such as "I don't know" and model refusals, should still be counted as the most RELEVANT.
- RESPONSE must be relevant to the entire QUESTION to get a score of 10.
- RELEVANCE score should increase as the RESPONSE provides RELEVANT context to more parts of the QUESTION.
- RESPONSE that is RELEVANT to none of the QUESTION should get a score of 0.
- RESPONSE that is RELEVANT to some of the QUESTION should get a score of 2–4.
- RESPONSE that is RELEVANT to most of the QUESTION should get a score between 5–8.
- RESPONSE that is RELEVANT and answers the entire QUESTION completely should get a score of 9 or 10.
- RESPONSE that is confidently FALSE should get a score of 0.
- RESPONSE that is only seemingly RELEVANT should get a score of 0.
- Never elaborate.
- Also return the reason behind the score.

Return output in JSON format containing:
{{
  "Score": <number>,
  "Reason": "<brief explanation>"
}}

QUESTION: {question}

RESPONSE: {response}

RELEVANCE:
Reason:
"""
    return prompt_template.strip()


In [ ]:
from tqdm import tqdm
import json
import pandas as pd

qa_scores = []
qa_reasons = []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🧠 Starting QA Relevance Evaluation...")
print(f"Total records: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        question = row.get("question", "")
        response = row.get("api_response_details.chat_summarization", "")

        if not question or not response:
            qa_scores.append(None)
            qa_reasons.append("Missing data in row")
            continue

        # Build prompt
        prompt = qa_relevance(question, response)

        # Call Azure GPT-4o
        completion = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "Return only valid JSON with keys 'Score' and 'Reason'."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        content = completion.choices[0].message.content.strip()
        content = content.replace("```json", "").replace("```", "").strip()

        # Parse safely
        try:
            parsed = json.loads(content)
            qa_scores.append(parsed.get("Score"))
            qa_reasons.append(parsed.get("Reason"))
        except Exception as e:
            qa_scores.append(None)
            qa_reasons.append(f"Invalid JSON output: {content}")

    except Exception as e:
        qa_scores.append(None)
        qa_reasons.append(f"Error: {e}")

# Add results
df["QA_Relevance_Score"] = qa_scores
df["QA_Relevance_Reason"] = qa_reasons

# Save results
output_path = "/content/evaluated_qa_relevance.xlsx"
df.to_excel(output_path, index=False)

print("\n✅ QA Relevance Evaluation Complete!")
print(f"📂 Saved to: {output_path}")
print(df[["question", "QA_Relevance_Score"]].head())


🧠 Starting QA Relevance Evaluation...
Total records: 10



100%|██████████| 10/10 [00:14<00:00,  1.47s/it]


✅ QA Relevance Evaluation Complete!
📂 Saved to: /content/evaluated_qa_relevance.xlsx
                                            question  QA_Relevance_Score
0  What is the background and epidemiology of chr...                   4
1  What tests are recommended for evaluating chro...                   9
2  What is the management approach for chronic st...                  10
3  What is the background and epidemiology of acr...                   4
4                       How is acromegaly diagnosed?                  10


##completeness

In [ ]:
# ====== Completeness Prompt Builder ======
def completeness_prompt(query, ground_truth, generated_answer):
    """
    Builds the completeness evaluation prompt for GPT.
    Compares generated answer vs ground truth for how fully it covers expected content.
    """
    return f"""
You are tasked with evaluating the **completeness** of a generated answer in response to a given query, using the provided ground truth as a reference.

---
**Query**: {query}
**Ground Truth**: {ground_truth}
**Generated Answer**: {generated_answer}
---

**Evaluation Objective**:
Assess whether the generated answer fully and adequately addresses all aspects of the query, and whether it incorporates all relevant information present in the ground truth.

---
**Evaluation Guidelines**:
- Determine if the answer responds to **all components** of the query (sub-questions, implied expectations).
- Check whether the answer includes **key facts, reasoning, and explanations** from the ground truth.
- Identify **omissions**, **partial responses**, or **underdeveloped sections**.
- Consider whether the answer is **logically complete** and **coherent**.

---
**Scoring Scale (1–5)**:
1 – Severely incomplete; major parts of the query are unaddressed.
2 – Largely incomplete; only a few aspects are covered.
3 – Moderately complete; some relevant information is missing.
4 – Mostly complete; minor omissions.
5 – Fully complete; all relevant aspects thoroughly addressed.

---
**Output Format (JSON Only)**:
{{
  "Completeness_Score": <1–5>,
  "Justification": "<Detailed reasoning explaining what was covered or missed>"
}}
"""


In [ ]:
# ====== Completeness Evaluation Runner ======
import json
from tqdm import tqdm

completeness_scores = []
completeness_reasons = []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🧩 Starting Completeness Evaluation...")
print(f"Total records: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        query = row.get("question", "") or row.get("Question", "")
        ground_truth = row.get("ground_truth_answer", "")
        generated = row.get("api_response_details.search_summarization", "")

        # Build prompt
        prompt = completeness_prompt(query, ground_truth, generated)

        # Send to Azure GPT-4o
        resp = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "Return ONLY valid JSON with keys 'Completeness_Score' and 'Justification'. No markdown or other text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=800
        )

        raw = resp.choices[0].message.content.strip()
        cleaned = raw.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(cleaned)

        completeness_scores.append(parsed.get("Completeness_Score"))
        completeness_reasons.append(parsed.get("Justification"))

    except Exception as e:
        completeness_scores.append(None)
        completeness_reasons.append(f"Error: {e}")

# Add new columns
df["Completeness_Score"] = completeness_scores
df["Completeness_Justification"] = completeness_reasons

# Save file (same master Excel)
output_file = "/content/evaluated_precision_recall_consistency_stability_relevance_completeness.xlsx"
df.to_excel(output_file, index=False)

print("\n✅ Completeness evaluation complete!")
print(f"📂 Results saved to: {output_file}")
print(df[["question", "Completeness_Score"]].head())


🧩 Starting Completeness Evaluation...
Total records: 10



100%|██████████| 10/10 [00:32<00:00,  3.27s/it]


✅ Completeness evaluation complete!
📂 Results saved to: /content/evaluated_precision_recall_consistency_stability_relevance_completeness.xlsx
                                            question  Completeness_Score
0  What is the background and epidemiology of chr...                   1
1  What tests are recommended for evaluating chro...                   1
2  What is the management approach for chronic st...                   1
3  What is the background and epidemiology of acr...                   1
4                       How is acromegaly diagnosed?                   1


##groundedness

In [ ]:
# ====== Groundedness Prompt Builder ======
def groundedness_prompt(ground_truth, generated_answer):
    """
    Builds the groundedness evaluation prompt to assess factual support
    of the generated answer relative to the ground truth.
    """
    return f"""
You are tasked with evaluating the **groundedness** of a generated answer using the provided ground truth as the factual reference.

---
**Ground Truth**: {ground_truth}
**Generated Answer**: {generated_answer}

---
**Evaluation Objective**:
Determine whether the generated answer is factually accurate and fully supported by the information present in the ground truth.

---
**Evaluation Guidelines**:
- Verify that **each factual claim** in the answer is **explicitly supported** by the ground truth.
- Identify any **hallucinated**, **fabricated**, or **unsupported statements**.
- Ensure the answer does not introduce **external knowledge** or **speculation** beyond the source content.
- Evaluate whether the answer maintains **factual fidelity**.

---
**Scoring Scale (1–5)**:
1 – Poorly grounded; multiple unsupported or fabricated claims.
2 – Weakly grounded; several inaccuracies.
3 – Partially grounded; some claims supported, others not.
4 – Mostly grounded; minor unsupported details.
5 – Fully grounded; all claims verifiable from the ground truth.

---
**Output Format (JSON Only)**:
{{
  "Groundedness_Score": <1–5>,
  "Justification": "<Explain which claims are supported or unsupported>"
}}
"""


In [ ]:
# ====== Groundedness Evaluation Runner ======
import json
from tqdm import tqdm

groundedness_scores = []
groundedness_reasons = []

with open("/content/dummy_medical_dataset.json", "r") as f:
    data = json.load(f)
df = pd.json_normalize(data["dataset"])

print("🔍 Starting Groundedness Evaluation...")
print(f"Total records: {len(df)}\n")

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        ground_truth = row.get("ground_truth_answer", "")
        generated = row.get("api_response_details.search_summarization", "")
        # Skip empty rows
        if not generated.strip() or not ground_truth.strip():
            groundedness_scores.append(None)
            groundedness_reasons.append("Skipped: Missing data")
            continue

        # Build prompt
        prompt = groundedness_prompt(ground_truth, generated)

        # Send to Azure GPT-4o
        resp = client.chat.completions.create(
            model=GPT4O_DEPLOYMENT_NAME,
            messages=[
                {"role": "system", "content": "Return ONLY valid JSON with keys 'Groundedness_Score' and 'Justification'. No markdown or extra text."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=800
        )

        raw = resp.choices[0].message.content.strip()
        cleaned = raw.replace("```json", "").replace("```", "").strip()
        parsed = json.loads(cleaned)

        groundedness_scores.append(parsed.get("Groundedness_Score"))
        groundedness_reasons.append(parsed.get("Justification"))

    except Exception as e:
        groundedness_scores.append(None)
        groundedness_reasons.append(f"Error: {e}")

# Add columns to DataFrame
df["Groundedness_Score"] = groundedness_scores
df["Groundedness_Justification"] = groundedness_reasons

# Save updated results
output_file = "/content/evaluated_precision_recall_consistency_relevance_completeness_groundedness.xlsx"
df.to_excel(output_file, index=False)

print("\n✅ Groundedness evaluation complete!")
print(f"📂 Results saved to: {output_file}")
print(df[["question", "Groundedness_Score"]].head())


🔍 Starting Groundedness Evaluation...
Total records: 10



100%|██████████| 10/10 [00:17<00:00,  1.78s/it]


✅ Groundedness evaluation complete!
📂 Results saved to: /content/evaluated_precision_recall_consistency_relevance_completeness_groundedness.xlsx
                                            question  Groundedness_Score
0  What is the background and epidemiology of chr...                   1
1  What tests are recommended for evaluating chro...                   1
2  What is the management approach for chronic st...                   1
3  What is the background and epidemiology of acr...                   1
4                       How is acromegaly diagnosed?                   1
